In [ ]:
import random
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Strategy constants
target_return_range = (0.3, 1.0)
loss_range = (-0.3, -0.7)

# Create interactive widgets
weeks_slider = widgets.IntSlider(value=12, min=4, max=52, step=1, description='Weeks:')
loss_chance_slider = widgets.FloatSlider(value=0.2, min=0.0, max=1.0, step=0.05, description='Loss Chance:')
initial_capital_slider = widgets.IntSlider(value=1000, min=100, max=10000, step=100, description='Initial Capital:')

def simulate_strategy(weeks, loss_chance, initial_capital):
    clear_output(wait=True)

    capital = initial_capital
    banked_profits = 0
    history = []

    for week in range(1, weeks + 1):
        week_data = {"Week": week, "Start Capital": capital, "Banked": banked_profits}

        if random.random() > loss_chance:
            weekly_return = random.uniform(*target_return_range)
            profit = capital * weekly_return
            reinvest_amount = profit * 0.5
            bank_amount = profit * 0.5
            capital += reinvest_amount
            banked_profits += bank_amount
            week_data.update({
                "Performance": f"+{weekly_return:.2%}",
                "Profit": profit,
                "Reinvested": reinvest_amount,
                "Banked This Week": bank_amount,
                "Loss": False
            })
        else:
            weekly_loss = random.uniform(*loss_range)
            loss = capital * abs(weekly_loss)
            capital -= loss
            if capital < initial_capital:
                capital += banked_profits
                banked_profits = 0
                week_data["Capital Boosted From Bank"] = True
            else:
                week_data["Capital Boosted From Bank"] = False
            week_data.update({
                "Performance": f"{weekly_loss:.2%}",
                "Profit": -loss,
                "Reinvested": 0,
                "Banked This Week": 0,
                "Loss": True
            })

        week_data["End Capital"] = capital
        history.append(week_data)

    df_sim = pd.DataFrame(history)

    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(df_sim['Week'], df_sim['End Capital'], marker='o', label='Capital')
    plt.plot(df_sim['Week'], df_sim['Banked'], marker='s', label='Banked Profits')
    plt.title('Weekly Capital & Banked Profits (Interactive Simulation)')
    plt.xlabel('Week')
    plt.ylabel('Amount ($)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Display final stats
    total_value = df_sim.iloc[-1]['End Capital'] + df_sim.iloc[-1]['Banked']
    print(f"Final Capital: ${df_sim.iloc[-1]['End Capital']:.2f}")
    print(f"Total Banked Profits: ${df_sim.iloc[-1]['Banked']:.2f}")
    print(f"Total Value (Capital + Banked): ${total_value:.2f}")

# Run the interactive widget
widgets.interact(simulate_strategy, 
                 weeks=weeks_slider, 
                 loss_chance=loss_chance_slider, 
                 initial_capital=initial_capital_slider)